In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
from xgboost import XGBClassifier
import os


In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd())# Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_NoDupsLessThemesENnoWords9.xlsx")
df = pd.read_excel(file_path)

#visualize the data
print(df.head())
print(df.info())

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jefva\\Documents\\Master\\Thesis_s2\\Data\\Grote_data_NoDupsLessThemesENnoWords9.xlsx'

In [ ]:
# Drop unnecessary columns
# Drop unnecessary columns
# Drop unnecessary columns
columns_to_drop = ["context","file_name","question","statistical"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# ✅ Drop rare themes (appearing < 2 times)
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 100].index
df = df[df["theme"].isin(valid_themes)]



# Amount of rows
print(f"Number of rows after filtering: {len(df)}")

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier


# 1. Encode labels
le = LabelEncoder()
y = le.fit_transform(df["theme"])
X = df["clean_text"]  

# 2. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# 3. TF-IDF vectorization with Dutch stopwords and bigrams
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")

dutch_stopwords = stopwords.words("dutch")

vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=10000,
    stop_words=dutch_stopwords
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:

# Minimal parameter grid for text data
param_grid = {
    'max_depth': [4],
    'learning_rate': [ 0.3],
    'n_estimators': [250],
}

xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='mlogloss',
    n_jobs=-1,
    verbosity=1,
    random_state=42
)

grid = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=3,
    verbose=2,
    n_jobs=-1
)

# 4. Fit (X_train must be raw text!)
print("GridSearchCV wordt uitgevoerd... 🚀")
grid.fit(X_train_vec, y_train)

# 5. Predict on test set
best_model = grid.best_estimator_

# 7. Predict + decode
y_pred = best_model.predict(X_test_vec)
y_test_labels = le.inverse_transform(y_test)
y_pred_labels = le.inverse_transform(y_pred)



print(f"Beste parameters: {grid.best_params_}")
print(classification_report(y_test, y_pred, zero_division=0))

# 7. Summary table
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="weighted", zero_division=1)

baseline_results = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1"],
    "Score": [accuracy, precision, recall, f1]
})

print(baseline_results)